In [ ]:
import os
from pathlib import Path
HOME = os.environ['HOME']

dataroot = Path(os.path.join(HOME, 'data/ciss'))
import gdal

In [ ]:
cd {dataroot}

In [ ]:
ls

In [ ]:
cd pdsdata_sRM_2015-03-12T18-29-13.808978-2/

In [ ]:
ls

In [ ]:
path = 'img_cal_cylmap_rot.cub'
ds = gdal.Open(path)

In [ ]:
img = ds.ReadAsArray()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,9)
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
img[img<0] = np.nan

In [ ]:
from IPython.html.widgets import interactive, interact, fixed
from IPython.display import display
from skimage import data, filter, io

In [ ]:
interpolators = ['none', 'nearest', 'bilinear', 'bicubic',
    'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser',
    'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc',
    'lanczos']
def myimshow(vmin, vmax, i):
    fig, ax = plt.subplots()
    ax.imshow(img[1010:1250, 550:549+1180], vmin=vmin, vmax=vmax, aspect='auto',
              interpolation=interpolators[i])
    ax.set_title('vmin: {:.2f}, vmax: {:.2f}, interpolator:{}'.format(vmin,vmax,
                                                                     interpolators[i]))
    plt.show()

In [ ]:
interact(myimshow, vmin=(0.06, 0.09, 0.001),
         vmax=(0.10, 0.12, 0.001), i=(0, len(interpolators)-1));

In [ ]:
subframe = img[1010:1250, 550:1180]

In [ ]:
plt.imshow(subframe, vmin=0.07, vmax=0.11, interpolation='sinc')

# Removal of density wave

First look at one azimuthal pixel.

In [ ]:
plt.plot(subframe[:, 250])

Possibly it will be better to get the mean value across the azimuth for a good density wave structure.

In [ ]:
azimuth_mean = subframe.mean(axis=1)
plt.plot(azimuth_mean)

See how the structures between the large peaks have been removed by the averaging.

In [ ]:
subframe.shape

In [ ]:
subtracted = subframe - azimuth_mean[:,np.newaxis]

In [ ]:
def show_img(img, vmin, vmax):
    plt.imshow(img, vmin=vmin, vmax=vmax, aspect='auto', interpolation='sinc')
    plt.title("{}, {}".format(vmin, vmax))
    plt.show()
    
interact(show_img, img=fixed(subtracted), vmin=(-0.007, 0, 0.001), vmax=(0., 0.021, 0.001));

In [ ]:
plt.imshow(subtracted, vmin=-0.004, vmax=0.003, interpolation='sinc',aspect='auto')
plt.savefig('real_waves_or_not.png', dpi=150)

In [ ]:
pwd

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction

In [ ]:
filtered = gaussian_filter(subframe, 1)

In [ ]:
plt.imshow(filtered)
plt.colorbar(orientation='horizontal')

In [ ]:
image = np.nan_to_num(filtered)

In [ ]:
seed = np.copy(image)

In [ ]:
seed[1:-1, 1:-1] = image.min()

In [ ]:
mask = image

In [ ]:
dilated = reconstruction(seed, mask, method='dilation')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(10,15))
ax1.imshow(image)
ax1.set_title('Original, but filtered')
ax1.axis('off')

ax2.imshow(dilated)
ax2.set_title('dilated')
ax2.axis('off')

ax3.imshow(image - dilated)
ax3.set_title('image - dilated')
ax3.axis('off')

fig.tight_layout()

In [ ]:
dilate_subtracted = image - dilated

In [ ]:
interact(show_img, img=fixed(dilate_subtracted),
         vmin=(-0.007, 0, 0.001), vmax=(0., 0.021, 0.001));

In [ ]:
plt.imshow(dilate_subtracted, vmin=-0.001, vmax=0.003, interpolation='sinc')

In [ ]:
from skimage.filter.rank import maximum, minimum, gradient

In [ ]:
from skimage.morphology import disk
from skimage import img_as_ubyte